In [4]:
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range 

In [5]:
pickle_file = "nlp/NLP_assignments/assignment-09/notMNIST.pickle"
with open(pickle_file, "rb") as f:
    save = pickle.load(f)
    train_dataset = save["train_dataset"]
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [17]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape(-1, image_size*image_size)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    # one-hot encoder
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)


Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


首先需要定义计算图 `with graph.as_default():`


In [20]:
train_subset = 10000

graph = tf.Graph()

with graph.as_default():
    tf_train_dataset = tf.constant(train_dataset[:train_subset,:])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    weights = tf.Variable(
        tf.truncated_normal([image_size*image_size,num_labels])
    )
    biases = tf.Variable(tf.zeros([num_labels]))
    
    logits = tf.matmul(tf_train_dataset, weights) + biases
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels= tf_train_labels, logits=logits)
    )
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases
    )
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset,weights)+biases)
    

In [23]:
num_steps = 1000
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if step % 100 == 0:
            print("Loss at step %d %d" %(step,l))
    print("test accuracy: %.1f%%" % accuracy(test_prediction.eval(),test_labels))


Initialized
Loss at step 0 17
Loss at step 100 2
Loss at step 200 1
Loss at step 300 1
Loss at step 400 1
Loss at step 500 1
Loss at step 600 1
Loss at step 700 1
Loss at step 800 1
Loss at step 900 1
test accurary: 82.8%


In [34]:
graph = tf.Graph()
with graph.as_default():
    features = tf.placeholder(tf.float32, shape=(None,image_size*image_size))
    labels = tf.placeholder(tf.float32, shape=(None,num_labels))
    weights = tf.Variable(tf.truncated_normal([image_size*image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    logits = tf.matmul(features, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=labels,logits= logits)
    )
    optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
    prediction = tf.nn.softmax(logits)

batch_size = 100
epoches = 400
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    l = 0
    num_steps = len(train_dataset) // batch_size +1
    for e in range(epoches):
        for step in range(num_steps):
            offset = (step *batch_size) %(train_labels.shape[0])
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            train_feed_dict = {features : batch_data, labels  : batch_labels}
            _, l, train_predictions = session.run(
          [optimizer, loss, prediction], feed_dict=train_feed_dict)
        if (e % 100 == 0):
            print("Mini-batch loss at epoch %d: %f" % (e, l))
    test_feed_dict = {features:test_dataset, labels:test_labels }
    test_prediction =session.run(prediction, feed_dict=test_feed_dict)
    print(accuracy(test_prediction,test_labels))


Initialized
Mini-batch loss at epoch 0: 3.509690
Mini-batch loss at epoch 100: 0.827261
Mini-batch loss at epoch 200: 0.710755
Mini-batch loss at epoch 300: 0.666889
Mini-batch loss at epoch 400: 0.643765
89.63


In [36]:
graph = tf.Graph()
layer_1 = 1024
with graph.as_default():
    features = tf.placeholder(tf.float32, shape=(None,image_size*image_size))
    labels = tf.placeholder(tf.float32, shape=(None,num_labels))
    weights_1 = tf.Variable(tf.truncated_normal([image_size*image_size, layer_1]))
    biases_1 = tf.Variable(tf.zeros([layer_1]))
    weights_2 = tf.Variable(tf.truncated_normal([layer_1, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    logits_1 = tf.matmul(features, weights_1) + biases_1
    relu_ = tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_, weights_2) + biases_2
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=labels,logits= logits_2)
    )
    optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
    prediction = tf.nn.softmax(logits_2)

batch_size = 200
epoches = 400
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    l = 0
    num_steps = len(train_dataset) // batch_size +1
    for e in range(epoches):
        for step in range(num_steps):
            offset = (step *batch_size) %(train_labels.shape[0])
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            train_feed_dict = {features : batch_data, labels  : batch_labels}
            _, l, train_predictions = session.run(
          [optimizer, loss, prediction], feed_dict=train_feed_dict)
        if (e % 100 == 0):
            print("Mini-batch loss at epoch %d: %f" % (e, l))
    test_feed_dict = {features:test_dataset, labels:test_labels }
    test_prediction =session.run(prediction, feed_dict=test_feed_dict)
    print(accuracy(test_prediction,test_labels))




Initialized
Mini-batch loss at epoch 0: 24.819849
Mini-batch loss at epoch 100: 0.352989
Mini-batch loss at epoch 200: 0.308277
Mini-batch loss at epoch 300: 0.284084
92.29
